# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:

Write a function to normalize features

Implement coordinate descent for LASSO

Explore effects of L1 penalty

In [1]:
import pandas as pd

dtype_dict = {'bathrooms':float, 'waterfront':int, 
              'sqft_above':int, 'sqft_living15':float,
              'grade':int, 'yr_renovated':int, 'price':float,
              'bedrooms':float, 'zipcode':str, 'long':float,
              'sqft_lot15':float, 'sqft_living':float, 'floors':float,
              'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 
              'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)


In [2]:
import numpy as np

def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    
    return(predictions)

In [3]:
def polynomial_dataframe(feature, degree):
    # assume that degree >= 1
    # initialize the SFrame:
    
    # and set poly_sframe['power_1'] equal to the passed feature
    poly_sframe = pd.DataFrame(feature)
    poly_sframe.columns = ['power_1']
    # first check if degree > 1
    if degree > 1:
        # then loop over the remaining degrees:
        # range usually starts at 0 and stops at the endpoint-1. We want it to start at 2 and stop at degree
        for power in range(2, degree+1): 
            # first we'll give the column a name:
            name = 'power_' + str(power)
            # then assign poly_sframe[name] to the appropriate power of feature
            poly_sframe[name] = feature**power
            
    return poly_sframe

In [4]:
sales['constant'] =1

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [5]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print(X)

[[  3.   5.   8.]
 [  4.  12.  15.]]


In [6]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print(norms)

[  5.  13.  17.]


In [7]:
print(X / norms) # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


In [7]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis =0)
    normalized_features = feature_matrix / norms
    
    return normalized_features, norms

In [9]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print(features)
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print(norms)
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function

```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:


In [8]:
simple_feature_matrix = sales[['constant','sqft_living','bedrooms']]
example_output = sales['price']

In [9]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of ro[i]:

In [10]:
weights = np.array([1., 4., 1.])

Use predict_output() to make predictions on this data.

In [11]:
prediction = predict_output(simple_feature_matrix, weights)
print(prediction.shape)

(21613,)


Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [14]:
ro = np.dot(example_output - prediction + np.dot(simple_feature_matrix, weights), simple_feature_matrix)

In [23]:
np.dot(simple_feature_matrix, weights)

print((example_output - prediction + np.dot(simple_feature_matrix, weights)).shape)
print(ro)

(21613,)
[ 79400304.63764465  87939472.68182787  80966703.40538493]


***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate?

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

In [24]:
simple_feature_matrix = sales[['constant','sqft_living','bedrooms']]
example_output = sales['price']
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)
simple_feature_matrix = simple_feature_matrix.values
weights = np.array([1., 4., 1.])

In [28]:
r0 =  np.dot(example_output - prediction + np.dot(simple_feature_matrix, weights), simple_feature_matrix)

print(r0)

[ 79400304.63764465  87939472.68182787  80966703.40538493]


In [29]:
# l1_penalty = 1.4e8
#weights_1_4e8, iteration = lasso_coordinate_descent_step(normalized_simple_feature_matrix, example_output,
#                                           initial_weights, 1.4e8, tolerance)
print(lasso_coordinate_descent_step(1, normalized_simple_feature_matrix, example_output, weights, 1.4e8))
print(lasso_coordinate_descent_step(2, normalized_simple_feature_matrix, example_output, weights, 1.4e8))


17939470.8233
10966698.6662


In [33]:
# l1_penalty = 1.64e8
#                                           initial_weights, 1.4e8, tolerance)
print(lasso_coordinate_descent_step(1, normalized_simple_feature_matrix, example_output, weights, 1.64e8))
print(lasso_coordinate_descent_step(2, normalized_simple_feature_matrix, example_output, weights, 1.64e8))

print('')
# l1_penalty = 1.73e8
print(lasso_coordinate_descent_step(1, normalized_simple_feature_matrix, example_output, weights, 1.73e8))
print(lasso_coordinate_descent_step(2, normalized_simple_feature_matrix, example_output, weights, 1.73e8))

print('')
# l1_penalty = 1.9e8
print(lasso_coordinate_descent_step(1, normalized_simple_feature_matrix, example_output, weights, 1.9e8))
print(lasso_coordinate_descent_step(2, normalized_simple_feature_matrix, example_output, weights, 1.9e8))

# l1_penalty = 2.3e8
print(lasso_coordinate_descent_step(1, normalized_simple_feature_matrix, example_output, weights, 2.3e8))
print(lasso_coordinate_descent_step(2, normalized_simple_feature_matrix, example_output, weights, 2.3e8))

5939470.82325
0.0

1439470.82325
0.0

0.0
0.0
0.0
0.0


## Single Coordinate Descent Step
Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [12]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i =  np.dot(output - prediction + feature_matrix[:,i] * weights[i], feature_matrix[:,i])
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i +  l1_penalty /2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty /2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [77]:
print(np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]))

[[ 0.83205029  0.31622777]
 [ 0.5547002   0.9486833 ]]


In [45]:

# should print 0.425558846691
import math
print(lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                                                 [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.425558846691


# Cyclical coordinate descent

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:

As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.
Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [14]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = initial_weights
    test = 0
    iteration = 0
    while(test == 0):
        test = 1
        iteration +=1
        for i in range(len(weights)):
            old_weights_i = weights[i]
            new_weights_i = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)   
            weights[i] = new_weights_i

            if abs(new_weights_i- old_weights_i) >= tolerance:
                test = 0
            else:
                test = test * 1
            #print("check test value: ", test)
    
        
    return weights, iteration
        

In [179]:
def lasso_cyclical_coordinate_descent_print_result(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = initial_weights
    test = 0
    iteration = 0
    while(test == 0):
        test = 1
        iteration +=1
        for i in range(len(weights)):
            old_weights_i = weights[i]
            new_weights_i = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)   
            weights[i] = new_weights_i

            if abs(new_weights_i- old_weights_i) >= tolerance:
                test = 0
            else:
                test = test * 1
            #print("check test value: ", test)
        print("iteration: ", iteration)
        print("weights: ", weights)
       
    return weights, iteration

Using the following parameters, learn the weights on the sales dataset.

In [15]:
simple_feature_matrix = sales[['constant','sqft_living','bedrooms']]
example_output = sales['price']
normalized_simple_feature_matrix, norms = normalize_features(simple_feature_matrix)
normalized_simple_feature_matrix = normalized_simple_feature_matrix.values
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, normalized_simple_feature_matrix.

Then, run your implementation of LASSO coordinate descent:

In [16]:
weights, iteration = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, example_output,
                                            initial_weights, l1_penalty, tolerance)

In [17]:
print("final Weight: ", weights)
print("Number of iteration: ", iteration)

final Weight:  [ 21624997.95951908  63157247.20788956         0.        ]
Number of iteration:  93


**QUIZ QUESTIONS**

What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
Which features had weight zero at convergence?

In [113]:
def RSS_model(weights, matrix_features, output):
    prediction = predict_output(matrix_features, weights)
    RSS = ((prediction - output)**2).sum()
    return RSS


In [115]:
print(RSS_model(weights, normalized_simple_feature_matrix, example_output)/1e14)

16.304924767153803


# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [122]:
testing = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict)
testing['constant'] =1
training['constant'] =1

let's consider the following set:

In [123]:
all_features = ['constant',
                'bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features. (Make you store the norms for the normalization, since we'll use them later)

In [124]:
feature_matrix = sales[all_features]
output = sales['price']
normalized_feature_matrix, norms = normalize_features(feature_matrix)
normalized_feature_matrix = normalized_feature_matrix.values
initial_weights = np.zeros(len(all_features))
l1_penalty = 1e7
tolerance = 1.0

In [125]:
weights, iteration = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

print("final Weight: ", weights)
print("Number of iteration: ", iteration)

final Weight:  [ 24964802.80942068         0.                 0.          56397533.51865718
         0.                 0.           3689656.56716452
   8630251.04703001         0.                 0.                 0.
         0.                 0.                 0.        ]
Number of iteration:  93


In [127]:
def print_weights_feature(weights, features):
    
    results = pd.DataFrame(weights)
    results.columns =['weights']
    results['feature']= all_features
    print(results)

In [128]:
print_weights_feature(weights, all_features)

         weights        feature
0   2.496480e+07       constant
1   0.000000e+00       bedrooms
2   0.000000e+00      bathrooms
3   5.639753e+07    sqft_living
4   0.000000e+00       sqft_lot
5   0.000000e+00         floors
6   3.689657e+06     waterfront
7   8.630251e+06           view
8   0.000000e+00      condition
9   0.000000e+00          grade
10  0.000000e+00     sqft_above
11  0.000000e+00  sqft_basement
12  0.000000e+00       yr_built
13  0.000000e+00   yr_renovated


Next, learn the weights with l1_penalty=1e8, on the training data. Initialize weights to all zeros, and set the tolerance=1. Call resulting weights weights1e8, you will need them later.

In [133]:
l1_penalty = 1e8
feature_matrix = sales[all_features]
output = sales['price']
normalized_feature_matrix, norms = normalize_features(feature_matrix)
normalized_feature_matrix = normalized_feature_matrix.values
initial_weights = np.zeros(len(all_features))
weights, iteration = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

print(iteration)
print_weights_feature(weights, all_features)

2
         weights        feature
0   7.940030e+07       constant
1   0.000000e+00       bedrooms
2   0.000000e+00      bathrooms
3   0.000000e+00    sqft_living
4   0.000000e+00       sqft_lot
5   0.000000e+00         floors
6   0.000000e+00     waterfront
7   0.000000e+00           view
8   0.000000e+00      condition
9   0.000000e+00          grade
10  0.000000e+00     sqft_above
11  0.000000e+00  sqft_basement
12  0.000000e+00       yr_built
13  0.000000e+00   yr_renovated


**QUIZ QUESTION**

What features had non-zero weight in this case?

Finally, learn the weights with ```l1_penalty=1e4```, on the training data. Initialize weights to all zeros, and set the ```tolerance=5e5```. Call resulting weights weights1e4, you will need them later. (This case will take quite a bit longer to converge than the others above.)

In [145]:
l1_penalty = 1e4
tolerance = 5e5
weights1e4, iteration = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

print(iteration)
print_weights_feature(weights1e4, all_features)

75
         weights        feature
0   3.290218e+07       constant
1  -2.104832e+07       bedrooms
2   1.172652e+07      bathrooms
3   1.127792e+08    sqft_living
4  -2.390976e+06       sqft_lot
5  -3.542257e+06         floors
6   6.956463e+06     waterfront
7   8.214963e+06           view
8   1.045765e+07      condition
9   2.565027e+07          grade
10 -2.788943e+07     sqft_above
11 -8.333938e+06  sqft_basement
12 -5.281293e+07       yr_built
13  3.513673e+06   yr_renovated


In [146]:
l1_penalty = 1e7
normalized_feature_matrix, norms = normalize_features(feature_matrix)
normalized_feature_matrix = normalized_feature_matrix.values
initial_weights = np.zeros(len(all_features))

weights1e7, iteration = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

print(iteration)
print_weights_feature(weights1e7, all_features)

18
         weights        feature
0   2.646160e+07       constant
1   0.000000e+00       bedrooms
2   0.000000e+00      bathrooms
3   5.498416e+07    sqft_living
4   0.000000e+00       sqft_lot
5   0.000000e+00         floors
6   3.680872e+06     waterfront
7   8.729617e+06           view
8   0.000000e+00      condition
9   0.000000e+00          grade
10  0.000000e+00     sqft_above
11  0.000000e+00  sqft_basement
12  0.000000e+00       yr_built
13  0.000000e+00   yr_renovated


In [147]:
l1_penalty = 1e8

normalized_feature_matrix, norms = normalize_features(feature_matrix)
normalized_feature_matrix = normalized_feature_matrix.values
initial_weights = np.zeros(len(all_features))

weights1e8, iteration = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

print(iteration)
print_weights_feature(weights1e8, all_features)

2
         weights        feature
0   7.940030e+07       constant
1   0.000000e+00       bedrooms
2   0.000000e+00      bathrooms
3   0.000000e+00    sqft_living
4   0.000000e+00       sqft_lot
5   0.000000e+00         floors
6   0.000000e+00     waterfront
7   0.000000e+00           view
8   0.000000e+00      condition
9   0.000000e+00          grade
10  0.000000e+00     sqft_above
11  0.000000e+00  sqft_basement
12  0.000000e+00       yr_built
13  0.000000e+00   yr_renovated


# Rescaling Learned Weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

In [157]:
weights_normalized = {'1e4': weights1e4/norms,'1e7': weights1e7/norms, '1e8': weights1e8/norms}
import pprint
pprint.pprint(weights_normalized)

{'1e4': array([  2.23803608e+05,  -4.09439994e+04,   3.54411654e+04,
         3.37402534e+02,  -3.68885849e-01,  -1.51646356e+04,
         5.44872235e+05,   6.97336122e+04,   2.04939298e+04,
         2.25229349e+04,  -9.62587182e+01,  -1.06970093e+02,
        -1.82241271e+02,   5.82307263e+01]),
 '1e7': array([  1.79994200e+05,   0.00000000e+00,   0.00000000e+00,
         1.64496539e+02,   0.00000000e+00,   0.00000000e+00,
         2.88308130e+05,   7.41023081e+04,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00]),
 '1e8': array([ 540088.14176653,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ])}


# Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [163]:
# you don't need to normalize test data set anymore

RSS_test_1e4 =  RSS_model(weights_normalized['1e4'], testing[all_features], testing['price'])
print(RSS_test_1e4/1e14)


2.250153766247389


In [164]:
RSS_test_1e7 =  RSS_model(weights_normalized['1e7'], testing[all_features], testing['price'])
print(RSS_test_1e7/1e14)

2.7252671457516944


In [165]:
RSS_test_1e8 =  RSS_model(weights_normalized['1e8'], testing[all_features], testing['price'])
print(RSS_test_1e8/1e14)

5.371458503528432
